In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os

full_data = pd.DataFrame()

for file in os.listdir('./Sales_Data/'):
    df = pd.read_csv('./Sales_Data/'+file)
    full_data = pd.concat([full_data,df])



In [2]:
full_data.isna().mean() * 100

Order ID            0.291678
Product             0.291678
Quantity Ordered    0.291678
Price Each          0.291678
Order Date          0.291678
Purchase Address    0.291678
dtype: float64

In [3]:
full_data = full_data.dropna()

print(full_data.shape)

full_data = full_data.drop(full_data[full_data['Order Date'] == 'Order Date'].index)

full_data['Order Date'] = pd.to_datetime(full_data['Order Date'])
full_data['Quantity Ordered'] = pd.to_numeric(full_data['Quantity Ordered'])
full_data['Price Each'] = pd.to_numeric(full_data['Price Each'])

full_data['Order Day'] = full_data['Order Date'].dt.day
full_data['Order Month'] = full_data['Order Date'].dt.month
full_data['Order Year'] = full_data['Order Date'].dt.year

full_data['Total Value'] = full_data['Quantity Ordered'] * full_data['Price Each']

full_data['City'] =  full_data['Purchase Address'].str.split(',').str[1].str.replace('City','')




(186305, 6)


/var/folders/1t/vz2x4n4n36v6r8p1dl1yh54w0000gn/T/ipykernel_42793/3569422104.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  full_data['Order Date'] = pd.to_datetime(full_data['Order Date'])


#### The best month in the year

In [4]:
grouped_data = full_data.groupby('Order Month')['Total Value'].sum()
grouped_data = grouped_data.sort_values(ascending=False)
the_best_month = grouped_data.index[0]
value = grouped_data.values[0]
value = "{:,}".format(value)

print(f"The best month in 2019 was {the_best_month} and we earned {value}")

full_data.shape

The best month in 2019 was 12 and we earned 4,557,905.42


(182735, 11)

#### The most product sold in the city

In [5]:
grouped_by_city = full_data.groupby('City')['Quantity Ordered'].sum()
grouped_by_city = grouped_by_city.sort_values(ascending=False)

city = grouped_by_city.index[0]
products_sold = grouped_by_city.values[0]

print(f'{city} sold {products_sold} - this is the biggest salles')


 San Francisco sold 49363 - this is the biggest salles


In [6]:
full_data.head()

products_type = full_data.groupby('Product')['Order ID'].count()
products_type = products_type.sort_values(ascending=False)

products_type

Product
USB-C Charging Cable          21523
Lightning Charging Cable      21308
AAA Batteries (4-pack)        20293
AA Batteries (4-pack)         20214
Wired Headphones              18552
Apple Airpods Headphones      15273
Bose SoundSport Headphones    13109
27in FHD Monitor               7386
iPhone                         6725
27in 4K Gaming Monitor         6122
34in Ultrawide Monitor         6058
Google Phone                   5433
Flatscreen TV                  4705
Macbook Pro Laptop             4641
ThinkPad Laptop                4052
20in Monitor                   4032
Vareebadd Phone                2020
LG Washing Machine              649
LG Dryer                        640
Name: Order ID, dtype: int64

In [20]:
line_chart_data = grouped_data.sort_index()


line_chart = px.line(line_chart_data,
                     title='Sales per month',
                     x = line_chart_data.index,
                     y = line_chart_data.values,
                     markers=True)

line_chart.show()


Order Month
1     1786511.29
2     2158127.48
3     2755969.40
4     3336376.42
5     3101881.04
6     2524464.99
7     2587444.91
8     2191698.31
9     2050361.26
10    3679254.16
11    3149785.09
12    4557905.42
Name: Total Value, dtype: float64


#### TOP 10 products sold

In [86]:

from turtle import bgcolor


bar_chart_px = px.bar(
    products_type.sort_values(ascending=False).head(10),
    color_discrete_sequence=["orange"],
    width = 1250,
    height = 900,
    orientation = 'h',
    
    )

bar_chart_px.update_traces(
    textposition = "inside",
    insidetextanchor = 'middle',
    texttemplate = "%{x}",
    textfont = dict(size = 12, color = 'white'),
    hovertemplate = 'Product %{x} - %{y} <extra></extra>'
)

bar_chart_px.update_layout(
    plot_bgcolor = "white",
    title = dict(
        text = 'Top 10 products',
        font = dict(color = 'red', size = 25)
        ),
    yaxis = dict(
        categoryorder = 'total ascending',
        ticksuffix = ' ',
        side = 'left',
        tickfont  = dict(size = 14, color = 'blue')
        ),
    xaxis = dict(
        visible = False
    ),
    showlegend = False,
    xaxis_title = dict(
        text = 'Sales',
        font = dict(color = 'green', size = 15)
        ),
    yaxis_title = dict(
        text = 'Category',
        font = dict(color = 'Purple', size = 15)
        ),
    hoverlabel = dict(
        bgcolor = 'red',
        font_size = 12
        )
    
)

bar_chart_px.show()